In [ ]:
import gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt


In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
#param
l = 0.98 # lambda
a = 0.1 # learning rate
eg = 1.0 # epsilon greedy
egMin = 0.05
maxBufferSize = 500000
batchSize = 64

In [ ]:
class Agent(tf.keras.Model):

  def __init__(self):
    super(Agent, self).__init__(name='mon_agent')
    # Define your layers here.
    #self.dense1 = tf.keras.layers.Dense(16, activation='relu', input_shape=(4,))
    self.dense1 = tf.keras.layers.Dense(8, activation='relu')
    #self.dense2 = tf.keras.layers.Dense(8, activation='relu')
    self.dense3 = tf.keras.layers.Dense(2, activation='relu')

  def call(self, inputs):
    tmp = self.dense1(inputs)
    #tmp = self.dense2(tmp)
    return self.dense3(tmp)


In [ ]:
class Coach(tf.keras.Model):
    def __init__(self, agent):
        super(Coach, self).__init__()
        self.agent = agent

    def call(self, inputs):
        [states, masks] = inputs
        #tf.print("states : ", states)
        #tf.print("masks : ", masks)
        return agent(states) * masks


In [ ]:
def showHistory(history) :
    #print(history.history.keys())
    plt.figure(figsize=(15.0,10.0))
    fig, axes = plt.subplots(nrows=1, ncols=2) 
    fig.set_size_inches(15.0, 7.0)         
    axes[0].plot(history.history['loss'], label="loss")
    axes[0].legend()
    axes[1].plot(history.history['accuracy'], label="accuracy")
    axes[1].legend()
    plt.show()


In [ ]:
agent = Agent()
agent.compile(optimizer=tf.keras.optimizers.Adam())
agent.build(tf.TensorShape([None,4]))
#agent.build(tf.TensorShape([None,2]))

coach = Coach(agent)
coach.compile(optimizer=tf.keras.optimizers.Adam(),
    loss="mse",
    metrics=['accuracy'])
coach([tf.keras.Input(shape=(4,)), tf.keras.Input(shape=(2,))])
#coach([tf.keras.Input(shape=(2,)), tf.keras.Input(shape=(2,))])


agent.summary()
coach.summary()

#print(agent([[0,0,0,0]]));


In [ ]:
memory = [] #{ "states":[], "masks":[], "actions":[], "values":[]}
loss = []
accuracy = []
for epi in range(1,100) :
    #memory = { "states":[], "masks":[], "actions":[], "values":[]}
    state = env.reset() #[2:]
    c0 = 0
    c1 = 0
    # * * * explore * * *
    print(epi)
    for step in range(1000): #epi * 100):
        env.render()
        #print(state)
        if (random.random() > eg) :
            c0 +=1
            Qs = agent([state])[0];
            action = np.argmax(Qs);
            Q = Qs[action]
            #print("State: ", state, " Qs: ", Qs.numpy(), " Action: ", action, " Q: ", Q.numpy())
        else :
            c1 +=1
            Qs = agent([state])[0];
            action = random.randrange(2)
            Q = Qs[action]
            #print("State: ", state, " Action: ", action, " Q: ", Q.numpy())

        mask = np.zeros(2)
        mask[action] = 1
        mem = { "s": state, "a": action, "m": mask}
        state, rewards, done, info = env.step(action)
        #state = state[2:]
        if done : rewards = 0.0
        mem["r"] = rewards
        mem["sp"] = state
        mem["d"] = done
        memory.append(mem)
        #input()
        print(mem)
        #Q0 = Q    
        #if not done : 
        #    #rewards -= 1.0
        #    Qs = agent([state])[0];
        #    #Q = Q + a * (rewards + l * np.max(Qs) - Q)
        #    Q = rewards + l * np.max(Qs)
        #    #Q = Q.numpy()
        #else :
        #    #rewards = -1.0
        #    rewards = 0.0
        #    #print(Q.numpy(), ' => ')
        #    Q = Q + a * (rewards - Q)
        #    #Q = 0.0
        #    print('.', end='')
        #    #if o0 == None : o0 = Q #.numpy()

        
        #print(rewards, done, " Q : ", Q0.numpy(), " => ", Q.numpy())
        #print("rewards: ", rewards, " Qs: ", Qs.numpy(), " Q: ", Q.numpy(), " mask: ", mask)
        
        #memory["values"].append(mask * Q) #.numpy())
        #memory["targets"].append( { "mask": mask, "action":action, "value":Q.numpy() } )
        if done : print('.', end=''); state = env.reset() #[2:]
        #if rewards == 0 : state = env.reset()
    
    
    #print(memory)
    #eg = max(egMin, eg*.9)
    #print("\r\neG: ", eg, "; ", c0,"-",c1,"/",c0+c1)
    
        if len(memory)>500 :
            while len(memory) > maxBufferSize :
                memory = random.sample(memory, maxBufferSize)
                #index = random.randrange(maxBufferSize)
                #del memory["states"][index]
                #del memory["masks"][index]
                #del memory["actions"][index]
                #del memory["values"][index]

            # * * * learn * * *

            batch = random.sample(memory, batchSize)
            bs = [ b["s"] for b in batch ]
            bm = [ b["m"] for b in batch ]
            by = [ b["m"] * ( b['r'] if b['d'] else b['r'] + a * np.max(agent([b['sp']])[0]) )
                   for b in batch ]

            history = coach.fit(
                [np.array(bs), np.array(bm)],
                np.array(by),
                verbose=0)
            loss.append(history.history['loss'])
            accuracy.append(history.history['accuracy'])
            #print(history.history['loss'])
    
    #plt.figure(figsize=(15.0,7.0))
    fig, axes = plt.subplots(nrows=1, ncols=2) 
    fig.set_size_inches(15.0, 7.0)         
    axes[0].plot(loss, label="loss")
    axes[0].legend()
    axes[1].plot(accuracy, label="accuracy")
    axes[1].legend()
    plt.show()
    loss = []
    accuracy = []

    #showHistory(history)
    
    #print(agent([s0])[0].numpy(), " <= ", o0)
    #break
    
    
    

In [ ]:
#env.close()
#agent.save_weights('CartPoleV1.tf')
#agent.load_weights('CartPoleV1.tf')
#history = coach.fit(
#    [np.array(memory["states"]), np.array(memory["masks"])],
#    np.array(memory["values"]),
#    epochs=10, batch_size=64, verbose=1)
#by = [ agent([b['sp']])          for b in batch ]
#print(by)
#np.array(by * bm),
#by = [ b["m"] * ( b['r'] if b['d'] else b['r'] + a * np.max(agent([b['sp']])[0]) )
#           for b in batch ]
#print(by, bm)
#print(pix)
def drawAgent(agent) :
    rx = np.arange(-0.24,0.24,0.2, np.float32)
    ry = np.arange(-2,2,0.2, np.float32)
    def agentDir(x,y) :
        [a ,b] = agent([[0.0, 0.0, x, y]])[0]
    pix = [
        #[ [[type(0.0), type(0.0), type(x), type(y)]] for x in rx]
        [ agent([[0.0, 0.0, x, y]])[0] for x in rx]
        for y in ry
        ]
    print(pix)
    return
    fig, ax = plt.subplots()
    fig.set_size_inches(15.0, 7.0)         
    im = ax.imshow(pix)
    ax.set_xticks(np.arange(len(range(-5,15))))
    ax.set_yticks(np.arange(len(range(-5,15))))
    ax.set_xticklabels(range(-5,15))
    ax.set_yticklabels(range(-5,15))
    for y in range(len(range(-5,15))):
        for x in range(len(range(-5,15))):
            text = ax.text(x, y, pix[y][x], ha="center", va="center", color="w")
    plt.show()

#print(pix)
drawAgent(agent)


In [ ]:
#eg = 0.05